## **Mapeamento Galp**

##### **___Informações___**

**Projeto:** Galp

**Autor:** Lucas Pimenta dos Santos Monteiro

**Data:** 24/02/2025

##### **___Objetivo___**

A ideia desse cógido é automatizar a visualização de uma imagem .svg gerada no Inkscape de acordo com o output do programa (report.csv) e os dados de entrada definidos (entry_data.csv).

Para isso foram criados rótulos no Inkscape que identificam cada elemento de texto a partir do inkscape:label. Cada label recebeu o nome equivalente a variável proveniente do report.csv e entry_data.csv que representa o valor desejado.

##### **Importando as Bibliotecas**

In [38]:
import pandas as pd
import xml.etree.ElementTree as ET

In [39]:
entry_data = pd.read_csv("entry_data.csv")
report = pd.read_csv("report.csv")

##### **Verificando os Dados**

In [40]:
entry_data.head()

,Entry data:,Unnamed: 1,Unnamed: 2
0,Entry feed mass flow rate =,600.0000000000,kg/h
1,Vacuum pressure =,-495.8300000000,mbar
2,Membrane area =,12.9600000000,m²
3,Membrane thickness =,100.0000000000,microns
4,Membrane porosity =,85.0000000000%,NaN


In [41]:
report.head()

,Time (s),Desal - Coolant mass flow rate (kg/h),Desal - Hot feedwater inlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Cold feedwater inlet temperature (°C),Desal - Cold feedwater outlet temperature (°C),Desal - Feed membrane temperature (°C),Desal - Membrane gap temperature (°C),Desal - Gap film boundary temperature (°C),Desal - Film wall temperature (°C),...,Electrolyzer - Electrical current (A),Electrolyzer - Heat transfer rate generated (W),Electrolyzer - Heat generated (kWh),Electrolyzer - Power demand (W),Electrolyzer - Electricity demand (kWh),Electrolyzer - Hydrogen flow rate (Nm³/h),Electrolyzer - Hydrogen produced (Nm³),Electrolyzer - Water consumption rate (kg/h),Electrolyzer - Water consumed (kg),Electrolyzer - Conversion efficiency (%)
0,126.0,600.0,60.002252,35.318482,30.173623,54.855866,47.164940,45.973937,44.702077,43.337278,...,59.619427,620.126044,0.021547,1517.220890,0.052809,0.299158,0.010425,2.504637,0.087280,70.076422
1,252.0,600.0,61.104548,35.449538,30.156997,55.810590,47.765583,46.550670,45.252720,43.835647,...,60.137350,629.118798,0.043410,1534.006836,0.106208,0.301757,0.020941,2.526395,0.175327,69.992852
2,378.0,600.0,62.154754,35.573148,30.142675,56.722993,48.337383,47.100869,45.779319,44.312469,...,60.647602,638.034895,0.065587,1550.600703,0.160191,0.304317,0.031548,2.547831,0.264130,69.910137
3,504.0,600.0,63.155680,35.689827,30.130347,57.595056,48.881935,47.625875,46.282945,44.768678,...,61.150235,646.872689,0.088074,1567.001629,0.214752,0.306839,0.042244,2.568947,0.353677,69.828280
4,630.0,600.0,64.110055,35.800057,30.119744,58.428755,49.400780,48.127004,46.764675,45.205220,...,61.645301,655.630588,0.110870,1583.208800,0.269884,0.309324,0.053027,2.589745,0.443958,69.747281


##### **Definindo as Colunas que serão utilizadas de cada dataframe**

In [42]:
userows_entry_data = [
    "Cooling heat exchanger flow rate =",
    "Brine flow rate out of the regulating tank =",
    "Brackish water temperature at the entrance of the regulating tank =",
    "Brackish water salinity at the entrance of the regulating tank =",
    "Temperature at the entrance of the cold side =",
    "Heating heat exchanger mass flow rate =",
    "Motor power demanded/generated ="
    "Electrolyzer cell temperature =",
    "Cathode pressure ="

]

usecols_report = [
"Time (s)",
"Desal - Total water produced (kg)",
"Desal - Distilled water production rate (kg/h)",
"FeedTank - Brackish water inlet flow rate (kg/h)",
"ColdHex - Heat transfer rate (W)",
"ColdHex - Cold side outlet temperature (°C)",
"Desal - Hot feedwater outlet temperature (°C)",
"Desal - Hot feedwater outlet mass flow rate (kg/h)",
"Desal - Hot feedwater outlet salinity (wt%)",
"HotHex - Cold side salinity (wt%)",
"ColdHex - Hot side outlet temperature (°C)",
"ColdHex - Hot side salinity (wt%)",
"HotHex - Cold side salinity (wt%)",
"HotHex - Heat transfer rate (W)",
"HotHex - Hot side outlet temperature (°C)",
"HotHex - Hot side inlet temperature (°C)",
"HotHex - Cold side outlet temperature (°C)",
"ColdHex - Hot side inlet temperature (°C)",
"ColdHex - Hot side inlet mass flow rate (kg/h)",
"HotHex - Cold side inlet temperature (°C)",
"Time (s)",
"SolarField - Solar irradiance (W/m²)", # segundo 11088 -> tempo de máximo
"SolarField - Efficiency",
"SolarField - Solar field inlet temperature (°C)",
"SolarField - Solar field outlet temperature (°C)",
"CircuitFromSolar - Outlet temperature (°C)",
"CircuitToSolar - Inlet temperature (°C)",
"ColdHex - Hot side salinity (wt%)",
"FeedTank - Water salinity (wt%)",
"FeedTank - Water temperature (°C)",
"Desal - Film wall temperature (°C)",
"Desal - Gap film boundary temperature (°C)",
"HCPV - Inlet temperature (°C)",
"HCPV - Outlet temperature (°C)",
"HCPV - Mass flow rate (kg/s)",
"HCPV - Heat transfer rate recovered from the solar cells (kW)",
"HCPV - Electrical power (kW)",
"HCPV - Global panel electrical efficiency (%)",
"HCPV - Electrical energy generated (kWh)",
"CircuitToSolar - Mass flow rate (kg/s)",
"ETC - Efficiency",
"ETC - ETC inlet temperature (°C)",
"ETC - ETC outlet temperature (°C)",
"ETC - Mass flow rate (kg/s)",
"PVT - PVT inlet temperature (°C)",
"PVT - Thermal Efficiency",
"PVT - Electrical power output (W)",
"PVT - Electrical Efficiency",
"PVT - Electrical energy generated (kWh)",
"PVT - PVT outlet temperature (°C)",
"PVT - Mass flow rate (kg/s)",
"PVT - Heat transfer rate (W)",
"ETC - Heat transfer rate (W)",
"SolarField - Heat transfer rate (W)",
"Reservoir - Temperature (°C)",
"Mix Valve - Mass flow rate in from reservoir(kg/s)",
"Mix Valve - Temperature in from reservoir(kg/s)",
"Mix Valve - Out Temperature (°C)",
"Mix Valve - Out Mass flow rate (kg/s)",
"Motor - Outlet mass flow rate (kg/s)",
"Motor - Outlet temperature motor (°C)",
"Motor - Radiator in mass flow rate (kg/s)",
"Motor - Radiator bypass mass flow rate (kg/s)",
"Motor - Recirculation mass flow rate (kg/s)",
"Motor - Nominal Mass flow rate (kg/s)",
"Motor - Mixture temperature engine (°C)",
"Motor - Mixture temperature radiator (°C)",
"Motor - Oulet temperature radiator (°C)",
"Motor - Heat coolant (kW)",
"Motor - System efficiency",
"Motor - Electrical energy generated/demanded (kWh)",
"Motor - Fuel consumed (L)",
"Motor - Fuel consumption (L/h)",
"Motor - Fuel consumption (g/kWh)",
"Electrolyzer - Hydrogen flow rate (Nm³/h)",
"Electrolyzer - Hydrogen produced (Nm³)",
"Electrolyzer - Water consumption rate (kg/h)",
"Electrolyzer - Water consumed (kg)",
"Electrolyzer - Conversion efficiency (%)",
"Electrolyzer - Heat transfer rate generated (W)",
"Electrolyzer - Power demand (W)",
"Electrolyzer - Electricity demand (kWh)"
]

##### Filtrando os Dataframes de Acordo com as Colunas

In [43]:
entry_data_filtrado = entry_data[entry_data['Entry data:'].isin(userows_entry_data)]
entry_data_filtrado.head()

,Entry data:,Unnamed: 1,Unnamed: 2
19,Cooling heat exchanger flow rate =,2000.0000000000,kg/h
20,Temperature at the entrance of the cold side =,30.0000000000,°C
23,Brackish water salinity at the entrance of the...,1.0000000000,wt%
24,Brackish water temperature at the entrance of ...,25.0000000000,°C
25,Brine flow rate out of the regulating tank =,0.0000000000,kg/h


In [44]:
report_filtrado = report.filter(items=usecols_report)
report_filtrado.head()

,Time (s),Desal - Total water produced (kg),Desal - Distilled water production rate (kg/h),FeedTank - Brackish water inlet flow rate (kg/h),ColdHex - Heat transfer rate (W),ColdHex - Cold side outlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Hot feedwater outlet mass flow rate (kg/h),Desal - Hot feedwater outlet salinity (wt%),HotHex - Cold side salinity (wt%),...,Motor - Fuel consumption (L/h),Motor - Fuel consumption (g/kWh),Electrolyzer - Hydrogen flow rate (Nm³/h),Electrolyzer - Hydrogen produced (Nm³),Electrolyzer - Water consumption rate (kg/h),Electrolyzer - Water consumed (kg),Electrolyzer - Conversion efficiency (%),Electrolyzer - Heat transfer rate generated (W),Electrolyzer - Power demand (W),Electrolyzer - Electricity demand (kWh)
0,126.0,0.601807,17.632586,17.632586,12868.585314,35.537275,35.318482,582.367414,1.034411,1.004012,...,5.950106,240.97929,0.299158,0.010425,2.504637,0.087280,70.076422,620.126044,1517.220890,0.052809
1,252.0,1.234167,18.490377,18.490377,11639.508135,35.008302,35.449538,581.509623,1.040287,1.008228,...,5.950106,240.97929,0.301757,0.020941,2.526395,0.175327,69.992852,629.118798,1534.006836,0.106208
2,378.0,1.896016,19.318024,19.318024,10580.611656,34.552590,35.573148,580.681976,1.046328,1.012640,...,5.950106,240.97929,0.304317,0.031548,2.547831,0.264130,69.910137,638.034895,1550.600703,0.160191
3,504.0,2.586309,20.116098,20.116098,9669.055750,34.160300,35.689827,579.883902,1.052530,1.017242,...,5.950106,240.97929,0.306839,0.042244,2.568947,0.353677,69.828280,646.872689,1567.001629,0.214752
4,630.0,3.304024,20.885317,20.885317,8885.033862,33.822904,35.800057,579.114683,1.058885,1.022027,...,5.950106,240.97929,0.309324,0.053027,2.589745,0.443958,69.747281,655.630588,1583.208800,0.269884


##### Separando apenas o tempo final do output gerado no report.csv

In [45]:
report_filtrado.tail()

,Time (s),Desal - Total water produced (kg),Desal - Distilled water production rate (kg/h),FeedTank - Brackish water inlet flow rate (kg/h),ColdHex - Heat transfer rate (W),ColdHex - Cold side outlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Hot feedwater outlet mass flow rate (kg/h),Desal - Hot feedwater outlet salinity (wt%),HotHex - Cold side salinity (wt%),...,Motor - Fuel consumption (L/h),Motor - Fuel consumption (g/kWh),Electrolyzer - Hydrogen flow rate (Nm³/h),Electrolyzer - Hydrogen produced (Nm³),Electrolyzer - Water consumption rate (kg/h),Electrolyzer - Water consumed (kg),Electrolyzer - Conversion efficiency (%),Electrolyzer - Heat transfer rate generated (W),Electrolyzer - Power demand (W),Electrolyzer - Electricity demand (kWh)
195,24696.0,214.694664,29.912625,29.912625,4397.289979,31.891831,37.078561,570.087375,2.558869,2.431298,...,5.950106,240.97929,0.218570,2.465610,1.829926,20.642781,66.070398,370.204843,1025.636174,13.247177
196,24822.0,215.737955,29.704461,29.704461,4388.973097,31.888253,37.055072,570.295539,2.565252,2.438253,...,5.950106,240.97929,0.214390,2.473186,1.794930,20.706213,66.096418,358.787179,1001.683660,13.282653
197,24948.0,216.773836,29.489312,29.489312,4380.075431,31.884425,37.030624,570.510688,2.571548,2.445159,...,5.950106,240.97929,0.210138,2.480615,1.759332,20.768409,66.122550,347.331557,977.477692,13.317285
198,25074.0,217.802068,29.267520,29.267520,4370.605153,31.880350,37.005241,570.732480,2.577755,2.452014,...,5.950106,240.97929,0.205812,2.487894,1.723115,20.829349,66.148775,335.841301,953.015764,13.351066
199,25200.0,218.822426,29.039416,29.039416,4360.570608,31.876032,36.978945,570.960584,2.583873,2.458816,...,5.950106,240.97929,0.201410,2.495020,1.686264,20.889010,66.175072,324.320019,928.295421,13.383987


In [46]:
report_filtrado = report_filtrado.iloc[[report_filtrado['SolarField - Solar irradiance (W/m²)'].idxmax()]].reset_index(drop=True)
report_filtrado.head()

,Time (s),Desal - Total water produced (kg),Desal - Distilled water production rate (kg/h),FeedTank - Brackish water inlet flow rate (kg/h),ColdHex - Heat transfer rate (W),ColdHex - Cold side outlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Hot feedwater outlet mass flow rate (kg/h),Desal - Hot feedwater outlet salinity (wt%),HotHex - Cold side salinity (wt%),...,Motor - Fuel consumption (L/h),Motor - Fuel consumption (g/kWh),Electrolyzer - Hydrogen flow rate (Nm³/h),Electrolyzer - Hydrogen produced (Nm³),Electrolyzer - Water consumption rate (kg/h),Electrolyzer - Water consumed (kg),Electrolyzer - Conversion efficiency (%),Electrolyzer - Heat transfer rate generated (W),Electrolyzer - Power demand (W),Electrolyzer - Electricity demand (kWh)
0,11088.0,93.745844,32.403412,32.403412,4477.253843,31.926237,37.280155,567.596588,1.71774,1.624972,...,5.950106,240.97929,0.408957,1.144865,3.423901,9.585134,65.576425,1050.044455,2276.395381,6.197443


##### Lendo o arquivo do Inkscape e definindo uma função de substituição de valores

In [47]:
# Caminho para o arquivo SVG
arquivo_svg = 'Diag_geral_2025_04_11.svg'

# Parse o arquivo SVG
tree = ET.parse(arquivo_svg)
root = tree.getroot()

# Definir namespaces
namespaces = {'svg': 'http://www.w3.org/2000/svg', 'inkscape': 'http://www.inkscape.org/namespaces/inkscape'}

def substituir_texto_por_label(label, novo_texto):
    # Encontre todos os elementos do tipo 'text'
    textos = root.findall('.//svg:text', namespaces)
    
    # Para cada elemento 'text' encontrado
    for texto in textos:
        # Verifique se o elemento 'text' possui o atributo 'inkscape:label'
        if texto.attrib.get('{http://www.inkscape.org/namespaces/inkscape}label') == label:
            # Itere sobre os elementos 'tspan' dentro do elemento 'text'
            for tspan in texto.findall('.//svg:tspan', namespaces):
                # Substitua o texto do 'tspan' pelo novo texto
                tspan.text = novo_texto

#### Definindo Função para Cálculo da Salinidade em g/l

Atualmente o código gera no report a salinidade em porcentagem de peso wt% (weight percent), porém a salinidade depende da densidade que é inversamente proporcional a temperatura, por isso é necessário o uso da função a seguir:

In [48]:
def salt_water_density(T, s):
    a0 = 9.999 * 10**2
    a1 = 2.034 * 10**-2
    a2 = -6.162 * 10**-3
    a3 = 2.261 * 10**-5
    a4 = -4.657 * 10**-8
    b0 = 8.020 * 10**2
    b1 = -2.001
    b2 = 1.677 * 10**-2
    b3 = -3.060 * 10**-5
    b4 = -1.613 * 10**-5

    result = (a0 + a1 * T + a2 * T**2 + a3 * T**3 + a4 * T**4 + 
              b0 * s + b1 * s * T + b2 * s * T**2 + 
              b3 * s * T**3 + b4 * s**2 * T**2)

    return result

#### Definindo cada a temperatura para cada Salinidade

In [49]:
# De cima para baixo no desenho e da esquerda para a direita, temos

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### HotHex - Cold side salinity (wt%) -> está presente na primeira e na segunda instância de salinidade, os valores são iguais em wt% mas distintos em g/L


# A temperatura da primeira instância (mais a esquerda) é a variável HotHex - Cold side outlet temperature (°C)

# No desenho, o nome dessa instância da salinidade foi alterado para HotHex - Cold side salinity 1 (wt%)

T_css1 = float(report_filtrado['HotHex - Cold side outlet temperature (°C)'].iloc[0])
S_css1 = float(report_filtrado['HotHex - Cold side salinity (wt%)'].iloc[0]/100)
css1_g_l = "s " + str(round(S_css1 * salt_water_density(T_css1, S_css1), 2)) + " g/l"

# A temperatura da segunda instância (mais a direita) é a variável HotHex - Cold side inlet temperature (°C)
# No desenho, o nome dessa instância da salinidade foi alterado para HotHex - Cold side salinity 2 (wt%)

T_css2 = float(report_filtrado['HotHex - Cold side inlet temperature (°C)'].iloc[0])
S_css2 = float(report_filtrado['HotHex - Cold side salinity (wt%)'].iloc[0]/100)
css2_g_l = "s " + str(round(S_css2 * salt_water_density(T_css2, S_css2), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### Desal - Hot feedwater outlet salinity (wt%) -> terceira instância de salinidade

# A temperatura da terceira instância está na variável -> Desal - Hot feedwater outlet temperature (°C)

T_hfos = float(report_filtrado['Desal - Hot feedwater outlet temperature (°C)'].iloc[0])
S_hfos = float(report_filtrado['Desal - Hot feedwater outlet salinity (wt%)'].iloc[0]/100)
hfos_g_l = "s " + str(round(S_hfos * salt_water_density(T_hfos, S_hfos), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### ColdHex - Hot side salinity (wt%) -> está presente na quarta e quinta instância de salinidade

# A temperatura da quarta instância (mais para cima) é a variável ColdHex - Hot side outlet temperature (°C)
# No desenho, o nome dessa instância da salinidade foi alterado para ColdHex - Hot side salinity 1 (wt%)

T_hss1 = float(report_filtrado['ColdHex - Hot side outlet temperature (°C)'].iloc[0])
S_hss1 = float(report_filtrado['ColdHex - Hot side salinity (wt%)'].iloc[0]/100)
hss1_g_l = "s " + str(round(S_hss1 * salt_water_density(T_hss1, S_hss1), 2)) + " g/l"

# A temperatura da quinta instância (mais para baixo) é a variável ColdHex - Hot side inlet temperature (°C)
# No desenho, o nome dessa instância da salinidade foi alterado para ColdHex - Hot side salinity 2 (wt%)

T_hss2 = float(report_filtrado['ColdHex - Hot side inlet temperature (°C)'].iloc[0])
S_hss2 = float(report_filtrado['ColdHex - Hot side salinity (wt%)'].iloc[0]/100)
hss2_g_l = "s " + str(round(S_hss2 * salt_water_density(T_hss2, S_hss2), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### Brackish water salinity at the entrance of the regulating tank = -> está presente na sexta instância de salinidade

# A temperatura da sexta instância está na variável -> Brackish water temperature at the entrance of the regulating tank =

T_bws = float(entry_data_filtrado[entry_data_filtrado['Entry data:'] == 'Brackish water temperature at the entrance of the regulating tank =']['Unnamed: 1'].iloc[0])
S_bws = float(entry_data_filtrado[entry_data_filtrado['Entry data:'] == 'Brackish water salinity at the entrance of the regulating tank =']['Unnamed: 1'].iloc[0])/100
bws_g_l = "s " + str(round(S_bws * salt_water_density(T_bws, S_bws), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### FeedTank - Water salinity (wt%) -> está presente na sétima instância de salinidade

# A temperatura da sétima instância está na variável -> FeedTank - Water temperature (°C)

T_fts = float(report_filtrado['FeedTank - Water temperature (°C)'].iloc[0])
S_fts = float(report_filtrado['FeedTank - Water salinity (wt%)'].iloc[0]/100)
fts_g_l = "s " + str(round(S_fts * salt_water_density(T_fts, S_fts), 2)) + " g/l"

#### Passando o Desal - Total water produced (kg) para litro -> vamos dividir o valor em kg por 0.998

In [50]:
total_water_produced_kg = report_filtrado['Desal - Total water produced (kg)'].iloc[0]
total_water_produced_l = str(round(total_water_produced_kg / 0.998, 2)) + " L"
print(total_water_produced_l)

93.93 L


#### Calculando o valor de (Desal - Film wall temperature (°C)+Desal - Gap film boundary temperature (°C))/2

In [51]:
film_wall_temp = report_filtrado['Desal - Film wall temperature (°C)'].iloc[0]
gap_film_boundary_temp = report_filtrado['Desal - Gap film boundary temperature (°C)'].iloc[0]
average_temp = "T "+ str(round((film_wall_temp + gap_film_boundary_temp) / 2, 2)) + " °C"

#### Calculando o valor de Motor - Radiator in mass flow rate (kg/s) + Motor - Radiator bypass mass flow rate (kg/s)

In [52]:
motor_flow_rate_combined = "ṁ " + str(round(report_filtrado['Motor - Radiator in mass flow rate (kg/s)'].iloc[0] + report_filtrado['Motor - Radiator bypass mass flow rate (kg/s)'].iloc[0], 2)) + " kg/s"
print(motor_flow_rate_combined)

ṁ 2.49 kg/s


##### Substituindo os Valores

In [53]:
for coluna, valor in report_filtrado.items():
    valor = valor.iloc[0]
    grandeza = ""
    if '(' in coluna and ')' in coluna:
        unidade = coluna[coluna.rfind('(') + 1:coluna.rfind(')')]
        unidade = unidade.strip()
    elif coluna == "SolarField - Efficiency" or coluna == "PVT - Thermal Efficiency" or coluna == "ETC - Efficiency" or coluna == "PVT - Electrical Efficiency" or coluna == "Motor - System efficiency":
        unidade = "%"
    else:
        unidade = ""
    if coluna == "PVT - Heat transfer rate (W)" or coluna == "SolarField - Heat transfer rate (W)" or coluna == "ETC - Heat transfer rate (W)" or coluna == "PVT - Electrical power output (W)" or coluna == "Electrolyzer - Heat transfer rate generated (W)" or coluna == "Electrolyzer - Power demand (W)":
        unidade = "kW"
        valor = "{:.2f}".format(float(valor/1000))
    if coluna == "HotHex - Heat transfer rate (W)" or coluna == "ColdHex - Heat transfer rate (W)":
        unidade = "kW"
        grandeza = "Q"
        valor = "{:.2f}".format(float(valor/1000))
    if unidade == '°C':
        grandeza = "T"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'kg/s':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'Kg/h' or unidade == 'kg/h':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'g/L':
        grandeza = "S"
        valor = "{:.2f}".format(float(valor))
    valor = "{:.2f}".format(float(valor))
    valor_novo = grandeza + " " + str(valor) + " " + unidade
    print(valor_novo)
    substituir_texto_por_label(coluna, valor_novo)

 11088.00 s
 93.75 kg
ṁ 32.40 kg/h
ṁ 32.40 kg/h
Q 4.48 kW
T 31.93 °C
T 37.28 °C
ṁ 567.60 kg/h
 1.72 wt%
 1.62 wt%
T 30.06 °C
 1.62 wt%
Q 4.95 kW
T 77.28 °C
T 77.72 °C
T 77.70 °C
T 36.61 °C
ṁ 600.00 kg/h
T 70.48 °C
 952.20 W/m²
 34.16 %
T 76.10 °C
T 79.17 °C
T 79.06 °C
T 76.27 °C
 1.62 wt%
T 36.61 °C
T 51.54 °C
T 53.73 °C
T 76.27 °C
T 69.49 °C
ṁ 0.34 kg/s
 -8.62 kW
 4.06 kW
 23.69 %
 11.05 kWh
ṁ 0.34 kg/s
 55.11 %
T 76.27 °C
T 79.03 °C
ṁ 0.23 kg/s
T 76.27 °C
 40.05 %
 0.49 kW
 13.73 %
 1.35 kWh
T 77.96 °C
ṁ 0.23 kg/s
 1.43 kW
 2.34 kW
 3.90 kW
T 76.27 °C
ṁ 0.50 kg/s
ṁ 76.27 kg/s
T 77.72 °C
ṁ 2.99 kg/s
ṁ 2.49 kg/s
T 78.01 °C
ṁ 0.09 kg/s
ṁ 2.40 kg/s
ṁ 2.04 kg/s
ṁ 4.53 kg/s
T 77.23 °C
T 76.59 °C
T 57.68 °C
 13.25 kW
 34.92 %
 61.60 kWh
 18.33 L
 5.95 L/h
 240.98 g/kWh
 0.41 Nm³/h
 1.14 Nm³
ṁ 3.42 kg/h
 9.59 kg
 65.58 %
 1.05 kW
 2.28 kW
 6.20 kWh


In [54]:
for linha in entry_data_filtrado.itertuples():
    grandeza = ""
    label = linha[1].rstrip(' ')
    valor = linha[2]
    valor = "{:.2f}".format(float(valor))
    unidade = linha[3].strip()
    if unidade == '°C':
        grandeza = "T"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'kg/s':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'kg/h' or unidade == 'Kg/h':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'g/L':
        grandeza = "S"
        valor = "{:.2f}".format(float(valor))

    valor_novo = grandeza + " " + str(valor) + ' ' + unidade
    #print(valor_novo)
    substituir_texto_por_label(label, str(valor_novo))

In [55]:
substituir_texto_por_label("HotHex - Cold side salinity 1 (wt%)", css1_g_l)
substituir_texto_por_label("HotHex - Cold side salinity 2 (wt%)", css2_g_l)
substituir_texto_por_label("Desal - Hot feedwater outlet salinity (wt%)", hfos_g_l)
substituir_texto_por_label("ColdHex - Hot side salinity 1 (wt%)", hss1_g_l)
substituir_texto_por_label("ColdHex - Hot side salinity 2 (wt%)", hss2_g_l)
substituir_texto_por_label("Brackish water salinity at the entrance of the regulating tank =", bws_g_l)
substituir_texto_por_label("FeedTank - Water salinity (wt%)", fts_g_l)
substituir_texto_por_label("Motor - Radiator in mass flow rate (kg/s) + Motor - Radiator bypass mass flow rate (kg/s)", motor_flow_rate_combined)

# correção do valor de total de água produzida
substituir_texto_por_label("Desal - Total water produced (kg)", total_water_produced_l)

# correção do valor de temperatura média
substituir_texto_por_label("(Desal - Film wall temperature (°C)+Desal - Gap film boundary temperature (°C))/2", average_temp)

##### Salvando o arquivo

In [56]:
# Salvar as alterações no arquivo SVG
tree.write('Diag_geral_modificado.svg')